In [1]:
#Importamos las librerías
import ee
import geemap

In [2]:
#Nos autenticamos y iniciamos ee
ee.Authenticate()

True

In [3]:
ee.Initialize()

In [4]:
#Creamos nuestro primer mapa para visualizar
Map = geemap.Map()

In [5]:
# 1. Definir un punto (Vertedero de Burgos)

point = ee.Geometry.Point([-3.663863749008054
, 42.31458303530395])

# Creamos un área de 10x10 metros alrededor del punto
# Buffer de 5 m -> bounds() -> rectángulo de 10m x 10m
aoi_10m = point.buffer(5).bounds()

# 2. Cargar Sentinel-2 Harmonized
s2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
      .filterDate("2025-01-01", "2025-10-01")   # rango de fechas
      .filterBounds(point)
      .filter(ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 5)))  # menos del 5% de nubes

img_cant =  s2.count().getInfo()  # Número de imágenes
# Sacamos la media imagen
image = s2.median().clip(aoi_10m)

# 3. Calcular reflectancias medias en el área 10x10 m

vals = image.reduceRegion(
    reducer=ee.Reducer.mean(),   
    geometry=aoi_10m,
    scale=10
).getInfo()

print("Reflectancias medias en área 10x10 m:", vals)


Reflectancias medias en área 10x10 m: {'AOT': 182.5, 'B1': 1017.9795321637428, 'B11': 4085.5760233918127, 'B12': 2951.0058479532163, 'B2': 1620.3040935672516, 'B3': 1946.7894736842106, 'B4': 2615.0643274853796, 'B5': 3077.9853801169593, 'B6': 3199.472222222222, 'B7': 3412.980994152047, 'B8': 3452.5877192982457, 'B8A': 3751.3918128654973, 'B9': 3543.9269005847955, 'MSK_CLASSI_CIRRUS': 0, 'MSK_CLASSI_OPAQUE': 0, 'MSK_CLASSI_SNOW_ICE': 0, 'MSK_CLDPRB': 0, 'MSK_SNWPRB': 0, 'QA10': None, 'QA20': None, 'QA60': 0, 'SCL': 5, 'TCI_B': 165.280701754386, 'TCI_G': 198.23684210526318, 'TCI_R': 249.65204678362574, 'WVP': 1343.7134502923977}


In [6]:
# Convertimos el punto a proyección UTM (EPSG:25830)
point_proj = point.transform('EPSG:25830', 1)

coords = point_proj.coordinates().getInfo()
x, y = coords  # en metros UTM

cell_size = 10  # metros

offsets = {
    "center": (0, 0),
    "N": (0, cell_size),
    "NE": (cell_size, cell_size),
    "E": (cell_size, 0),
    "SE": (cell_size, -cell_size),
    "S": (0, -cell_size),
    "SW": (-cell_size, -cell_size),
    "W": (-cell_size, 0),
    "NW": (-cell_size, cell_size)
}

results_dict = {}

for name, (dx, dy) in offsets.items():
    shifted = ee.Geometry.Point([x + dx, y + dy], 'EPSG:25830')
    aoi = shifted.buffer(5).bounds()

    vals = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi,
        scale=10,
        crs='EPSG:25830',
        maxPixels=1e9
    ).getInfo()
    
    results_dict[name] = vals


In [7]:
print(results_dict)

{'center': {'AOT': 182.5, 'B1': 1019.7569169960475, 'B11': 4100.683794466404, 'B12': 2993.916996047431, 'B2': 1646.7628458498025, 'B3': 1993.604743083004, 'B4': 2695.086956521739, 'B5': 3100.9644268774705, 'B6': 3234.448616600791, 'B7': 3447.195652173913, 'B8': 3539.505928853755, 'B8A': 3782.4960474308305, 'B9': 3593.693675889328, 'MSK_CLASSI_CIRRUS': 0, 'MSK_CLASSI_OPAQUE': 0, 'MSK_CLASSI_SNOW_ICE': 0, 'MSK_CLDPRB': 0, 'MSK_SNWPRB': 0, 'QA10': None, 'QA20': None, 'QA60': 0, 'SCL': 5, 'TCI_B': 168.08498023715413, 'TCI_G': 203.07509881422925, 'TCI_R': 251.99802371541503, 'WVP': 1346.887351778656}, 'N': {'AOT': 182.5, 'B1': 1028.5, 'B11': 4175, 'B12': 3205, 'B2': 1653.7578947368422, 'B3': 2047.105263157895, 'B4': 2827, 'B5': 3214, 'B6': 3406.5, 'B7': 3615.5, 'B8': 3697.9684210526316, 'B8A': 3935.5, 'B9': 3838.5, 'MSK_CLASSI_CIRRUS': 0, 'MSK_CLASSI_OPAQUE': 0, 'MSK_CLASSI_SNOW_ICE': 0, 'MSK_CLDPRB': 0, 'MSK_SNWPRB': 0, 'QA10': None, 'QA20': None, 'QA60': 0, 'SCL': 5, 'TCI_B': 168.87894736

In [ ]:
# 1. Definir un punto (Vertedero de Burgos)
long = -3.663863749008054
lat = 42.31458303530395

# Convertir 10 metros a grados aproximadamente
# 1 grado de latitud ≈ 111,000 metros
# 1 grado de longitud ≈ 111,000 * cos(latitud) metros
cell_size_lat = 10 / 111000  # ~0.00009 grados
cell_size_long = 10 / (111000 * abs(lat * 3.14159 / 180))  # ajustado por latitud


offsets = {
    "center": (long, lat),
    "N": (long, lat + cell_size_lat),
    "NE": (long + cell_size_long, lat + cell_size_lat),
    "E": (long + cell_size_long, lat),
    "SE": (long + cell_size_long, lat - cell_size_lat),
    "S": (long, lat - cell_size_lat),
    "SW": (long - cell_size_long, lat - cell_size_lat),
    "W": (long - cell_size_long, lat),
    "NW": (long - cell_size_long, lat + cell_size_lat)
}

point = ee.Geometry.Point(offsets["center"])

# 2. Cargar Sentinel-2 Harmonized
s2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
      .filterDate("2025-01-01", "2025-10-01")   # rango de fechas
      .filterBounds(point)
      .filter(ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 5)))  # menos del 5% de nubes

image = s2.first()
print("Imagen usada:", image.get("PRODUCT_ID").getInfo())

results_dict = {}

for name, coords in offsets.items():
    # Creamos un punto y area para cada offset
    point = ee.Geometry.Point(coords)
    aoi = point.buffer(5).bounds()

    # Extraemos las reflectancias medias
    vals = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi,
        scale=10
    ).getInfo()
    
    results_dict[name] = vals
    print(f"Offset {name} ({coords[0]:.8f}, {coords[1]:.8f}): {vals}")

KeyError: 'center'